# Task 4: Privacy and Fairness

In this section the goal is to train a classifier that is both private and fair and compare the results with previous ones. 

1. We apply **Local Differential Privacy** to the sensitive attributes (`Sex` and `Age`).
2. We try to reduce bias by **Reweighing** on this noisy data.
3. Finally, we check the model using the real data to see if the fairness mitigation was effective.


In [ ]:
# IMPORTS

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing.reweighing import Reweighing
import joblib

# Set seed for reproducibility
np.random.seed(42)

print("Starting Task 4: Private + Fair Classifier...")

# ==========================================
# 1. Load & Basic Prep
# ==========================================

cols = [
    'Age', 'Workclass', 'fnlwgt', 'Education', 'Education-Num',
    'Marital Status', 'Occupation', 'Relationship', 'Race', 'Sex',
    'Capital Gain', 'Capital Loss', 'Hours per week', 'Country', 'Target'
]

# Try loading from typical filenames
try:
    df = pd.read_csv('adult.data', header=None, names=cols, skipinitialspace=True)
except FileNotFoundError:
    df = pd.read_csv('adult.data.csv', header=None, names=cols, skipinitialspace=True)

# Clean target
df['Target'] = df['Target'].str.strip()
df['target_binary'] = (df['Target'] == '>50K').astype(int)

print(f"Data loaded: {df.shape}")

In [ ]:


# ==========================================
# 2. Local Differential Privacy
# ==========================================

def randomized_response(values, epsilon, categories):
    """
    Applies k-ary randomized response.
    """
    values = pd.Series(values)
    n = len(values)
    k = len(categories)
    
    # Probability of preserving the true value
    p = np.exp(epsilon) / (np.exp(epsilon) + k - 1)
    
    noisy_vals = []
    for v in values:
        # Handle missing or unknown categories by picking random
        if v not in categories or pd.isna(v):
            noisy_vals.append(np.random.choice(categories))
            continue
            
        # Flip coin based on epsilon
        if np.random.rand() < p:
            noisy_vals.append(v)
        else:
            # Pick from the other categories
            others = [c for c in categories if c != v]
            noisy_vals.append(np.random.choice(others))
            
    return np.array(noisy_vals)

# Create Age bins (needed for the LDP step, same as Q3)
bins = [0, 30, 40, 50, 100]
labels = ["<=30", "31-40", "41-50", "51+"]
df["AgeGroup"] = pd.cut(df["Age"], bins=bins, labels=labels, right=True, include_lowest=True)

# LDP Params
epsilon = 1.0
age_cats = labels
sex_cats = df["Sex"].unique().tolist()

print(f"Applying LDP noise (epsilon={epsilon})...")

# Create the noisy columns
df["AgeGroup_noisy"] = randomized_response(df["AgeGroup"], epsilon, age_cats)
df["Sex_noisy"] = randomized_response(df["Sex"], epsilon, sex_cats)

# ==========================================
# 3. Reconstruct Private Features
# ==========================================

# Map noisy categorical data back to numeric/binary for the model

# 1. Private Sex (1 = Noisy Male)
df["Sex_priv_Male"] = (df["Sex_noisy"] == "Male").astype(int)

# 2. Private Age (using midpoints of the bins)
age_map = {"<=30": 25, "31-40": 35, "41-50": 45, "51+": 60}
df["Age_priv"] = df["AgeGroup_noisy"].map(age_map).astype(float)

# Binarize Private Age for AIF360 logic
# We use >30 as the cutoff for 'privileged' to keep it simple based on our bins
df['age_priv_binary'] = (df['Age_priv'] > 30).astype(int)

# ==========================================
# 4. Feature Matrix Setup
# ==========================================

# Drop original sensitive cols, intermediates, and targets
cols_to_drop = ['Age', 'Sex', 'AgeGroup', 'AgeGroup_noisy', 'Sex_noisy', 'Target', 'target_binary']
cat_features = ['Workclass', 'Education', 'Marital Status', 'Occupation', 'Relationship', 'Race', 'Country']

# One-hot encoding
df_encoded = pd.get_dummies(
    df.drop(columns=cols_to_drop + ['age_priv_binary', 'Age_priv', 'Sex_priv_Male']), 
    columns=cat_features, 
    drop_first=True
)

# Re-attach the *private* features
X = df_encoded.copy()
X['Age_priv'] = df['Age_priv']
X['Sex_priv_Male'] = df['Sex_priv_Male']
# Aux binary col for AIF360
X['age_priv_binary'] = df['age_priv_binary']

y = df['target_binary']

print(f"Features ready. Count: {X.shape[1]}")

# ==========================================
# 5. Fairness Mitigation (Reweighing)
# ==========================================

# Split first to prevent leakage
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Setup AIF360 dataset (Training data only)
df_train_aif = X_train.copy()
df_train_aif['Target'] = y_train.values

# Define groups based on NOISY attributes
# Privileged: Noisy Male & Noisy Older
priv_groups = [{'Sex_priv_Male': 1, 'age_priv_binary': 1}]
unpriv_groups = [{'Sex_priv_Male': 0, 'age_priv_binary': 0}]

dataset_aif = BinaryLabelDataset(
    df=df_train_aif,
    label_names=['Target'],
    protected_attribute_names=['Sex_priv_Male', 'age_priv_binary'],
    favorable_label=1,
    unfavorable_label=0
)

print("Calculating weights via Reweighing (on noisy data)...")
RW = Reweighing(unprivileged_groups=unpriv_groups, privileged_groups=priv_groups)
RW.fit(dataset_aif)
dataset_transf = RW.transform(dataset_aif)

# Get the new weights
sample_weights = dataset_transf.instance_weights

# ==========================================
# 6. Train Model
# ==========================================

print("Training Random Forest with privacy & fairness weights...")

clf_priv_fair = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# Fit using the private features and the calculated weights
clf_priv_fair.fit(X_train, y_train, sample_weight=sample_weights)

print("Done. Model trained.")

# Save artifacts for later use
joblib.dump(clf_priv_fair, 'private_fair_classifier.joblib')
joblib.dump((X_test, y_test), 'test_data_q4.joblib')

Starting Task 4: Private + Fair Classifier...
Data loaded: (32561, 16)
Applying LDP noise (epsilon=1.0)...
Features ready. Count: 101
Calculating weights via Reweighing (on noisy data)...
Training Random Forest with privacy & fairness weights...
Done. Model trained.


['test_data_q4.joblib']

In [3]:
from aif360.metrics import ClassificationMetric

print(">>> Starting Fairness Audit using Real Data...")

# ==========================================
# 1. Prepare Audit Dataframe
# ==========================================

# Pull original rows corresponding to the test set to get real sensitive attrs
df_eval = df.loc[X_test.index].copy()

# Add Model Predictions (based on private features) and True Targets
df_eval['y_pred'] = clf_priv_fair.predict(X_test)
df_eval['y_true'] = y_test.values

# Reconstruct REAL binary sensitive attributes (Ground Truth, no noise)
df_eval['sex_real'] = (df_eval['Sex'] == 'Male').astype(int)

# Real Age Binary (using the original median)
median_age = df['Age'].median()
df_eval['age_real'] = (df['Age'] > median_age).astype(int)

# Subset only numeric columns for AIF360 to avoid ValueError
audit_cols = ['y_true', 'y_pred', 'sex_real', 'age_real']
df_aif = df_eval[audit_cols].copy()

# ==========================================
# 2. Setup AIF360 Datasets
# ==========================================

# Define groups based on REAL attributes
# Privileged: Real Men & Real Older > Median
priv_groups = [{'sex_real': 1, 'age_real': 1}]
unpriv_groups = [{'sex_real': 0, 'age_real': 0}]

# Ground Truth Dataset
ds_true = BinaryLabelDataset(
    df=df_aif.drop(columns=['y_pred']),
    label_names=['y_true'],
    protected_attribute_names=['sex_real', 'age_real'],
    favorable_label=1,
    unfavorable_label=0
)

# Prediction Dataset (swap target col with prediction col)
df_pred_temp = df_aif.copy()
df_pred_temp['y_true'] = df_pred_temp['y_pred'] # Overwrite label with prediction
df_pred_temp.drop(columns=['y_pred'], inplace=True)

ds_pred = BinaryLabelDataset(
    df=df_pred_temp,
    label_names=['y_true'],
    protected_attribute_names=['sex_real', 'age_real'],
    favorable_label=1,
    unfavorable_label=0
)

# ==========================================
# 3. Compute & Report Metrics
# ==========================================

metric = ClassificationMetric(
    ds_true, 
    ds_pred,
    unprivileged_groups=unpriv_groups,
    privileged_groups=priv_groups
)

print("-" * 50)
print("AUDIT RESULTS (Private + Fair Classifier)")
print("-" * 50)
print(f"Accuracy:                {metric.accuracy():.4f}")
print(f"Disparate Impact:        {metric.disparate_impact():.4f} (Ideal: 1.0)")
print(f"Statistical Parity Diff: {metric.statistical_parity_difference():.4f} (Ideal: 0.0)")
print(f"Equal Opportunity Diff:  {metric.equal_opportunity_difference():.4f} (Ideal: 0.0)")
print("-" * 50)

>>> Starting Fairness Audit using Real Data...
--------------------------------------------------
AUDIT RESULTS (Private + Fair Classifier)
--------------------------------------------------
Accuracy:                0.8454
Disparate Impact:        0.1684 (Ideal: 1.0)
Statistical Parity Diff: -0.2969 (Ideal: 0.0)
Equal Opportunity Diff:  -0.0187 (Ideal: 0.0)
--------------------------------------------------
